In [1]:
import pandas as pd
import numpy as np
import json 
import re
from pandas import json_normalize
from datetime import datetime
from sklearn.preprocessing import LabelEncoder


In [82]:
train1_df=pd.read_csv("data_given/train.csv")
#test_df=pd.read_csv('data_given/test.csv')



/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [60]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   channelGrouping       903653 non-null  object
 1   date                  903653 non-null  int64 
 2   device                903653 non-null  object
 3   fullVisitorId         903653 non-null  object
 4   geoNetwork            903653 non-null  object
 5   sessionId             903653 non-null  object
 6   socialEngagementType  903653 non-null  object
 7   totals                903653 non-null  object
 8   trafficSource         903653 non-null  object
 9   visitId               903653 non-null  int64 
 10  visitNumber           903653 non-null  int64 
 11  visitStartTime        903653 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 82.7+ MB


In [3]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804684 entries, 0 to 804683
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   channelGrouping       804684 non-null  object
 1   date                  804684 non-null  int64 
 2   device                804684 non-null  object
 3   fullVisitorId         804684 non-null  object
 4   geoNetwork            804684 non-null  object
 5   sessionId             804684 non-null  object
 6   socialEngagementType  804684 non-null  object
 7   totals                804684 non-null  object
 8   trafficSource         804684 non-null  object
 9   visitId               804684 non-null  int64 
 10  visitNumber           804684 non-null  int64 
 11  visitStartTime        804684 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 73.7+ MB


In [61]:
def date_process(df):
    df["date"] = pd.to_datetime(df["date"], format="%Y%m%d") # seting the column as pandas datetime
    df["weekday"] = df['date'].dt.weekday #extracting week day
    df["day"] = df['date'].dt.day # extracting day
    df["month"] = df['date'].dt.month # extracting month
    df["year"] = df['date'].dt.year # extracting year
    df['visitHour'] = (df['visitStartTime'].apply(lambda x: str(datetime.fromtimestamp(x).hour))).astype(int)
    
    return df
train_df=date_process(train_df)
#test_df=date_process(test_df)



In [83]:
#2.Function to validate the json columns in the dataset    
def column_validator(df):
    cols=[]
    for col in df.columns:
        if( type(df[col].values[0]) == str ):
            txt = df[col].values[0]
            if re.search("^{.*}$",txt):
                cols.append(col)
    return cols  # Returns the json columns

#2.1 Function for flattening the json columns and merge them with original dataset
def json_to_df(df,json_columns):
    for column in json_columns:
        column_to_df= json_normalize([json.loads(x) for x in df[column]])
        df= df.drop(column, axis=1).merge(column_to_df, right_index=True, left_index=True) # drop the flattened column from the original dataset
    return df ## returns new dataframe with flattened json columns


train1_json_columns = column_validator(train1_df)
#test_json_columns = column_validator(test_df)

if train_json_columns is not None:
    train1_df=json_to_df(train1_df,train_json_columns)
    
#if test_json_columns is not None:    
 #   test_df=json_to_df(test_df,test_json_columns)




In [84]:
train1_df.head()

channelGrouping      date        fullVisitorId  \
0  Organic Search  20160902  1131660440785968503   
1  Organic Search  20160902   377306020877927890   
2  Organic Search  20160902  3895546263509774583   
3  Organic Search  20160902  4763447161404445595   
4  Organic Search  20160902    27294437909732085   

                        sessionId  socialEngagementType     visitId  \
0  1131660440785968503_1472830385  Not Socially Engaged  1472830385   
1   377306020877927890_1472880147  Not Socially Engaged  1472880147   
2  3895546263509774583_1472865386  Not Socially Engaged  1472865386   
3  4763447161404445595_1472881213  Not Socially Engaged  1472881213   
4    27294437909732085_1472822600  Not Socially Engaged  1472822600   

   visitNumber  visitStartTime     browser                 browserVersion  \
0            1      1472830385      Chrome  not available in demo dataset   
1            1      1472880147     Firefox  not available in demo dataset   
2            1      1472865386      Chrome  not available in demo dataset   
3            1      1472881213  UC Browser  not available in demo dataset   
4            2      1472822600      Chrome  not available in demo dataset   

   ... adwordsClickInfo.criteriaParameters isTrueDirect referralPath  \
0  ...       not available in demo dataset          NaN          NaN   
1  ...       not available in demo dataset          NaN          NaN   
2  ...       not available in demo dataset          NaN          NaN   
3  ...       not available in demo dataset          NaN          NaN   
4  ...       not available in demo dataset         True          NaN   

   adwordsClickInfo.page adwordsClickInfo.slot adwordsClickInfo.gclId  \
0                    NaN                   NaN                    NaN   
1                    NaN                   NaN                    NaN   
2                    NaN                   NaN                    NaN   
3                    NaN                   NaN                    NaN   
4                    NaN                   NaN                    NaN   

  adwordsClickInfo.adNetworkType adwordsClickInfo.isVideoAd adContent  \
0                            NaN                        NaN       NaN   
1                            NaN                        NaN       NaN   
2                            NaN                        NaN       NaN   
3                            NaN                        NaN       NaN   
4                            NaN                        NaN       NaN   

  campaignCode  
0          NaN  
1          NaN  
2          NaN  
3          NaN  
4          NaN  

[5 rows x 55 columns]

In [85]:
train1_df['transactionRevenue']

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
903648    NaN
903649    NaN
903650    NaN
903651    NaN
903652    NaN
Name: transactionRevenue, Length: 903653, dtype: object

In [64]:
def remove_nan_cols(df):
    for col in df.columns:
        if (df[col].isnull().sum() >  (0.5 * len(df))):
            df.drop(col,axis=1,inplace=True)
    return df
train_df=remove_nan_cols(train_df)
#test_df=remove_nan_cols(test_df)
 

In [66]:
def impute_na(df):
    for col in df.columns:
        df[col].fillna(0,inplace=True)
    return df

train_df=impute_na(train_df)
#test_df=impute_na(test_df)

In [67]:
train_df.head()

channelGrouping       date        fullVisitorId  \
0  Organic Search 2016-09-02  1131660440785968503   
1  Organic Search 2016-09-02   377306020877927890   
2  Organic Search 2016-09-02  3895546263509774583   
3  Organic Search 2016-09-02  4763447161404445595   
4  Organic Search 2016-09-02    27294437909732085   

                        sessionId  socialEngagementType     visitId  \
0  1131660440785968503_1472830385  Not Socially Engaged  1472830385   
1   377306020877927890_1472880147  Not Socially Engaged  1472880147   
2  3895546263509774583_1472865386  Not Socially Engaged  1472865386   
3  4763447161404445595_1472881213  Not Socially Engaged  1472881213   
4    27294437909732085_1472822600  Not Socially Engaged  1472822600   

   visitNumber  visitStartTime  weekday  day  ...  \
0            1      1472830385        4    2  ...   
1            1      1472880147        4    2  ...   
2            1      1472865386        4    2  ...   
3            1      1472881213        4    2  ...   
4            2      1472822600        4    2  ...   

                       longitude                networkLocation  visits hits  \
0  not available in demo dataset  not available in demo dataset       1    1   
1  not available in demo dataset  not available in demo dataset       1    1   
2  not available in demo dataset  not available in demo dataset       1    1   
3  not available in demo dataset  not available in demo dataset       1    1   
4  not available in demo dataset  not available in demo dataset       1    1   

  pageviews newVisits   campaign  source   medium  \
0         1         1  (not set)  google  organic   
1         1         1  (not set)  google  organic   
2         1         1  (not set)  google  organic   
3         1         1  (not set)  google  organic   
4         1         0  (not set)  google  organic   

  adwordsClickInfo.criteriaParameters  
0       not available in demo dataset  
1       not available in demo dataset  
2       not available in demo dataset  
3       not available in demo dataset  
4       not available in demo dataset  

[5 rows x 48 columns]

In [68]:
train_df.drop('sessionId',axis=1,inplace=True) # Removing this column as  it is the  combination of fullVisitorId and visitId
train_df.drop('visitStartTime',axis=1,inplace=True)
train_df.drop('fullVisitorId',axis=1,inplace=True) # This column is very long and of no much contribution towards target variable

In [69]:
#5.Changing datatypes from object to desired ones
def data_type_convert(df):
    for col in df.columns:
        if ( type(df[col][0]) == str and df[col][0].isdigit()):
                df[col]=df[col].astype(int)
    return df

train_df=data_type_convert(train_df)
#test_df=data_type_convert(test_df)

In [70]:
def remove_zero_std_cols(df):
    for column in df.columns:
        if (df[column].nunique()==1):
            df.drop(column,axis=1,inplace=True)
    return df

train_df=remove_zero_std_cols(train_df)
#test_df=remove_zero_std_cols(test_df)



In [71]:
train_df.head()

channelGrouping       date     visitId  visitNumber  weekday  day  month  \
0  Organic Search 2016-09-02  1472830385            1        4    2      9   
1  Organic Search 2016-09-02  1472880147            1        4    2      9   
2  Organic Search 2016-09-02  1472865386            1        4    2      9   
3  Organic Search 2016-09-02  1472881213            1        4    2      9   
4  Organic Search 2016-09-02  1472822600            2        4    2      9   

   year  visitHour     browser  ...                         region  \
0  2016         21      Chrome  ...                          Izmir   
1  2016         10     Firefox  ...  not available in demo dataset   
2  2016          6      Chrome  ...            Community of Madrid   
3  2016         11  UC Browser  ...  not available in demo dataset   
4  2016         18      Chrome  ...  not available in demo dataset   

                           metro                           city  \
0                      (not set)                          Izmir   
1  not available in demo dataset  not available in demo dataset   
2                      (not set)                         Madrid   
3  not available in demo dataset  not available in demo dataset   
4  not available in demo dataset  not available in demo dataset   

     networkDomain hits pageviews newVisits   campaign  source   medium  
0     ttnet.com.tr    1         1         1  (not set)  google  organic  
1      dodo.net.au    1         1         1  (not set)  google  organic  
2  unknown.unknown    1         1         1  (not set)  google  organic  
3  unknown.unknown    1         1         1  (not set)  google  organic  
4  unknown.unknown    1         1         0  (not set)  google  organic  

[5 rows x 26 columns]

In [53]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_test_df=scaler.fit_transform(test_df)

In [54]:
scaled_test_df=pd.DataFrame(scaled_test_df)

In [55]:
scaled_test_df.head()

0         1         2         3         4         5         6   \
0 -0.197664 -0.043904  0.796076 -0.862119  0.605026 -0.531073 -0.532823   
1 -0.197664 -0.149929  0.796076 -0.862119  1.540704 -0.531073  0.889469   
2 -0.197664 -0.149929  0.796076 -0.862119  0.337689 -0.531073 -0.532823   
3 -0.197664 -0.149929  0.796076 -0.862119 -0.998994  1.890280  1.277367   
4 -0.197664 -0.149929  0.796076 -0.862119 -0.196984  1.890280 -0.532823   

         7         8         9   ...        14        15        16        17  \
0 -0.765970 -0.720074  0.381990  ... -2.345843  0.416190 -0.029538  0.082422   
1 -0.765970 -0.720074  1.497034  ...  0.723958  0.779062  0.092458  0.255262   
2 -0.765970 -0.720074  1.497034  ...  0.745100  0.883673  0.336450  0.600942   
3  1.305534  1.022280 -0.733055  ... -0.582610 -1.074751  0.458446  0.082422   
4 -0.765970 -0.720074 -0.733055  ...  0.106615 -1.074751  0.580442  0.082422   

         18        19        20        21        22        23  
0 -1.736984 -1.047365 -0.206321 -0.163222  0.355535 -0.294202  
1  0.575711 -1.047365 -0.206321 -0.163222  0.355535 -0.294202  
2  0.575711 -1.047365 -0.206321 -0.163222  0.355535 -0.294202  
3  0.575711 -1.047365 -0.206321 -0.163222  0.355535 -0.294202  
4  0.575711 -1.047365 -0.206321 -0.163222  0.355535 -0.294202  

[5 rows x 24 columns]

In [73]:
#8 Function to gather categorical columns in the dataset
def categorical_cols(df):
    cat_cols=[]
    for col in df.columns:
        if (type(df[col][0]) == str or type(df[col][0]) == np.bool_) :
            cat_cols.append(col)
    return cat_cols # returns categorical columns in the dataset
    
train_cat_cols=categorical_cols(train_df)   # collecting all categorical columns in the train dataset 
#test_cat_cols=categorical_cols(test_df)     # collecting all categorical columns in the test dataset

#8.1 Function for encoding categorical values to numerical values

def label_encoding(cat_cols,df):
    # creating instance of labelencoder
    labelencoder = LabelEncoder()
    # Assigning numerical values and storing in same column
    for column in cat_cols:
        df[column] = labelencoder.fit_transform(df[column].astype(str))
    return df

train_df=label_encoding(train_cat_cols,train_df)
#test_df=label_encoding(test_cat_cols,test_df) 

In [74]:
train_df.head()

channelGrouping       date     visitId  visitNumber  weekday  day  month  \
0                4 2016-09-02  1472830385            1        4    2      9   
1                4 2016-09-02  1472880147            1        4    2      9   
2                4 2016-09-02  1472865386            1        4    2      9   
3                4 2016-09-02  1472881213            1        4    2      9   
4                4 2016-09-02  1472822600            2        4    2      9   

   year  visitHour  browser  ...  region  metro  city  networkDomain  hits  \
0  2016         21       11  ...     150      0   258          24932     1   
1  2016         10       16  ...     375     93   648           6673     1   
2  2016          6       11  ...      78      0   325          25841     1   
3  2016         11       46  ...     375     93   648          25841     1   
4  2016         18       11  ...     375     93   648          25841     1   

   pageviews  newVisits  campaign  source  medium  
0          1          1         0     149       5  
1          1          1         0     149       5  
2          1          1         0     149       5  
3          1          1         0     149       5  
4          1          0         0     149       5  

[5 rows x 26 columns]

In [76]:
from sklearn.impute import KNNImputer
imputer=KNNImputer()

#imputer_test_df=imputer.fit_transform(test_df[['pageviews']]) ## imputing pageviews with KNNimputer in test data
#test_df['pageviews']=imputer_test_df

imputer_train_df=imputer.fit_transform(train_df[['pageviews']]) ## imputing pageviews with KNNimputer in test data
train_df['pageviews']=imputer_train_df



In [77]:
train_df.head()

channelGrouping       date     visitId  visitNumber  weekday  day  month  \
0                4 2016-09-02  1472830385            1        4    2      9   
1                4 2016-09-02  1472880147            1        4    2      9   
2                4 2016-09-02  1472865386            1        4    2      9   
3                4 2016-09-02  1472881213            1        4    2      9   
4                4 2016-09-02  1472822600            2        4    2      9   

   year  visitHour  browser  ...  region  metro  city  networkDomain  hits  \
0  2016         21       11  ...     150      0   258          24932     1   
1  2016         10       16  ...     375     93   648           6673     1   
2  2016          6       11  ...      78      0   325          25841     1   
3  2016         11       46  ...     375     93   648          25841     1   
4  2016         18       11  ...     375     93   648          25841     1   

   pageviews  newVisits  campaign  source  medium  
0        1.0          1         0     149       5  
1        1.0          1         0     149       5  
2        1.0          1         0     149       5  
3        1.0          1         0     149       5  
4        1.0          0         0     149       5  

[5 rows x 26 columns]

In [39]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804684 entries, 0 to 804683
Data columns (total 28 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   channelGrouping  804684 non-null  int64         
 1   date             804684 non-null  datetime64[ns]
 2   visitId          804684 non-null  int64         
 3   visitNumber      804684 non-null  int64         
 4   weekday          804684 non-null  int64         
 5   day              804684 non-null  int64         
 6   month            804684 non-null  int64         
 7   year             804684 non-null  int64         
 8   visitHour        804684 non-null  int64         
 9   browser          804684 non-null  int64         
 10  operatingSystem  804684 non-null  int64         
 11  isMobile         804684 non-null  int64         
 12  deviceCategory   804684 non-null  int64         
 13  continent        804684 non-null  int64         
 14  subContinent     804

In [6]:
 ## Stages of Data Preprocessing and Data transformation ##


#1.Function for extracting features from date column

def date_process(df):
    df["date"] = pd.to_datetime(df["date"], format="%Y%m%d") # seting the column as pandas datetime
    df["weekday"] = df['date'].dt.weekday #extracting week day
    df["day"] = df['date'].dt.day # extracting day
    df["month"] = df['date'].dt.month # extracting month
    df["year"] = df['date'].dt.year # extracting year
    df['visitHour'] = (df['visitStartTime'].apply(lambda x: str(datetime.fromtimestamp(x).hour))).astype(int)
    
    return df
train_df=date_process(train_df)
test_df=date_process(test_df)

#2.Function to validate the json columns in the dataset    
def column_validator(df):
    cols=[]
    for col in df.columns:
        if( type(df[col].values[0]) == str ):
            txt = df[col].values[0]
            if re.search("^{.*}$",txt):
                cols.append(col)
    return cols  # Returns the json columns

#2.1 Function for flattening the json columns and merge them with original dataset
def json_to_df(df,json_columns):
    for column in json_columns:
        column_to_df= json_normalize([json.loads(x) for x in df[column]])
        df= df.drop(column, axis=1).merge(column_to_df, right_index=True, left_index=True) # drop the flattened column from the original dataset
    return df ## returns new dataframe with flattened json columns


train_json_columns = column_validator(train_df)
test_json_columns = column_validator(test_df)

if train_json_columns is not None:
    train_df=json_to_df(train_df,train_json_columns)
    
if test_json_columns is not None:    
    test_df=json_to_df(test_df,test_json_columns)


#3.Dropping columns which have more than 50% of null values and not contributing to the target variable

def remove_nan_cols(df)
    for col in df.columns:
        if (df[col].isnull().sum() >  (0.5 * len(df))):
            df.drop(col,axis=1,inplace=True)
    return None
remove_nan_cols(train_df)
remove_nan_cols(test_df)
 

#4.Imputation of null values 
def impute_na(df):
    for col in df.columns:
        df[col].fillna(0.inplace=True)
    return None

train_df=impute_na(train_df)
test_df=impute_na(test_df)

train_df['adwordsClickInfo.isVideoAd'].fillna(True, inplace=True) # filling boolean with True
train_df[train_df['city'] == "(not set)"]['city'] = np.nan
train_df['city'].fillna("NaN", inplace=True)
    

test_df['isTrueDirect'].fillna(False, inplace=True) 
test_df['adwordsClickInfo.isVideoAd'].fillna(True, inplace=True) # filling boolean with True
test_df[test_df['city'] == "(not set)"]['city'] = np.nan
test_df['city'].fillna("NaN", inplace=True)   

#5.Changing datatypes from object to desired ones
def data_type_convert(df):
    for col in df.columns:
        if ( type(df[col][0]) == str and df[col][0].isdigit() ):
            print(col)
            df[col]=df[col].astype(int)

data_type_convert(train_df)
data_type_convert(test_df)

#6. Removing columns with constant values or with zero standard deviation
def remove_zero_std_cols(df):
    for column in df.columns:
        if (df[column].nunique()==1):
            df.drop(column,axis=1,inplace=True)
    return df

train_df=remove_zero_std_cols(train_df)
test_df=remove_zero_std_cols(test_df)


#7. sessionId col is removed as it is unique id and does'nt contribute to the target variable
train_df.drop('sessionId',axis=1,inplace=True)
test_df.drop('sessionId',axis=1,inplace=True)
train_df.drop('visitStartTime',axis=1,inplace=True) #removing visitStartime as it was extraced into visitHour
test_df.drop('visitStartTime',axis=1,inplace=True)


#8 Function to gather categorical columns in the dataset
def categorical_cols(df):
    cat_cols=[]
    for col in df.columns:
        if (type(df[col][0]) == str or type(df[col][0]) == np.bool_) :
            cat_cols.append(col)
    return cat_cols # returns categorical columns in the dataset
    
train_cat_cols=categorical_cols(train_df)   # collecting all categorical columns in the train dataset 
test_cat_cols=categorical_cols(test_df)     # collecting all categorical columns in the test dataset

#8.1 Function for encoding categorical values to numerical values

def label_encoding(cat_cols,df):
    # creating instance of labelencoder
    labelencoder = LabelEncoder()
    # Assigning numerical values and storing in same column
    for column in cat_cols:
        df[column] = labelencoder.fit_transform(df[column].astype(str))
    return df

label_encoding(train_cat_cols,train_df)
label_encoding(test_cat_cols,test_df) 

#9. Imputing pageviews column with KNNImputer in both train and test data

from sklearn.impute import KNNImputer
imputer=KNNImputer()

imputer_train_df=imputer.fit_transform(train_df[['pageviews']]) ## Imputing pageviews with KNNimputer in training data
train_df['pageviews']=imputer_train_df

imputer_test_df=imputer.fit_transform(test_df[['pageviews']]) ## imputing pageviews with KNNimputer in test data
test_df['pageviews']=imputer_test_df



In [78]:
dropped_cols=['visitId','weekday','day']
train_df.drop(dropped_cols,axis=1,inplace=True)

In [79]:
train_df.head()

channelGrouping       date  visitNumber  month  year  visitHour  browser  \
0                4 2016-09-02            1      9  2016         21       11   
1                4 2016-09-02            1      9  2016         10       16   
2                4 2016-09-02            1      9  2016          6       11   
3                4 2016-09-02            1      9  2016         11       46   
4                4 2016-09-02            2      9  2016         18       11   

   operatingSystem  isMobile  deviceCategory  ...  region  metro  city  \
0               16         0               0  ...     150      0   258   
1                7         0               0  ...     375     93   648   
2               16         0               0  ...      78      0   325   
3                6         0               0  ...     375     93   648   
4                1         1               1  ...     375     93   648   

   networkDomain  hits  pageviews  newVisits  campaign  source  medium  
0          24932     1        1.0          1         0     149       5  
1           6673     1        1.0          1         0     149       5  
2          25841     1        1.0          1         0     149       5  
3          25841     1        1.0          1         0     149       5  
4          25841     1        1.0          0         0     149       5  

[5 rows x 23 columns]

In [80]:
 train_df["date"] = pd.to_datetime(train_df["date"]).dt.date

In [95]:
train1_df['transactionRevenue'].fillna(0,inplace=True)

In [111]:
train1_df['transactionRevenue']=train1_df['transactionRevenue'].astype(float)

In [121]:
train_df=pd.concat([train_df,train1_df['transactionRevenue']],axis=1)

In [112]:
t=np.log1p(train1_df['transactionRevenue'].values)



In [122]:
#10. Model Training begins here 

import datetime

X = train_df[train_df['date']<=datetime.date(2017,5,31)] ## train data for the months till May 31 2017
val_X = train_df[train_df['date']>datetime.date(2017,5,31)] ## validation data for the months of june 2017 and july 2017

x = X.drop(['date'], axis = 1)
val_X = train_df.drop(['date'], axis = 1)



In [123]:
y = np.log1p(X["transactionRevenue"].values)
val_y = np.log1p(val_X["transactionRevenue"].values)

In [124]:
y = pd.DataFrame(y)
val_y = pd.DataFrame(val_y)

In [156]:
val_y

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 765707 entries, 0 to 765706
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       765707 non-null  float64
dtypes: float64(1)
memory usage: 5.8 MB


In [125]:
x = X.drop(['transactionRevenue'], axis = 1)
val_x = val_X.drop(['transactionRevenue'], axis = 1)

In [133]:
val_x.columns

Index(['channelGrouping', 'visitNumber', 'month', 'year', 'visitHour',
       'browser', 'operatingSystem', 'isMobile', 'deviceCategory', 'continent',
       'subContinent', 'country', 'region', 'metro', 'city', 'networkDomain',
       'hits', 'pageviews', 'newVisits', 'campaign', 'source', 'medium'],
      dtype='object')

In [135]:
x.drop('transactionRevenue',axis=1,inplace=True)

In [129]:
x = X.drop(['date'], axis = 1)
val_X = train_df.drop(['date'], axis = 1)




In [157]:
val_X1 = train_df[train_df['date']>datetime.date(2017,5,31)]

In [158]:
val_y = np.log1p(val_X1["transactionRevenue"].values)

In [160]:
val_y=pd.DataFrame(val_y)

In [161]:
val_y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137946 entries, 0 to 137945
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       137946 non-null  float64
dtypes: float64(1)
memory usage: 1.1 MB


In [147]:
val_X.drop('transactionRevenue',axis=1,inplace=True)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [152]:
val_X.drop('date',axis=1,inplace=True)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [153]:
val_X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 137946 entries, 4822 to 901262
Data columns (total 22 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   channelGrouping  137946 non-null  int64  
 1   visitNumber      137946 non-null  int64  
 2   month            137946 non-null  int64  
 3   year             137946 non-null  int64  
 4   visitHour        137946 non-null  int64  
 5   browser          137946 non-null  int64  
 6   operatingSystem  137946 non-null  int64  
 7   isMobile         137946 non-null  int64  
 8   deviceCategory   137946 non-null  int64  
 9   continent        137946 non-null  int64  
 10  subContinent     137946 non-null  int64  
 11  country          137946 non-null  int64  
 12  region           137946 non-null  int64  
 13  metro            137946 non-null  int64  
 14  city             137946 non-null  int64  
 15  networkDomain    137946 non-null  int64  
 16  hits             137946 non-null  i

In [151]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 765707 entries, 0 to 903652
Data columns (total 22 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   channelGrouping  765707 non-null  int64  
 1   visitNumber      765707 non-null  int64  
 2   month            765707 non-null  int64  
 3   year             765707 non-null  int64  
 4   visitHour        765707 non-null  int64  
 5   browser          765707 non-null  int64  
 6   operatingSystem  765707 non-null  int64  
 7   isMobile         765707 non-null  int64  
 8   deviceCategory   765707 non-null  int64  
 9   continent        765707 non-null  int64  
 10  subContinent     765707 non-null  int64  
 11  country          765707 non-null  int64  
 12  region           765707 non-null  int64  
 13  metro            765707 non-null  int64  
 14  city             765707 non-null  int64  
 15  networkDomain    765707 non-null  int64  
 16  hits             765707 non-null  int6

In [140]:
%%time
import xgboost as xg 
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score

model = xg.XGBRegressor(objective ='reg:squarederror', n_estimators = 1000,learning_rate=0.5,max_depth=8)
model.fit(x, y) 

CPU times: user 1h 8min 16s, sys: 22.1 s, total: 1h 8min 38s
Wall time: 18min 28s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.5, max_delta_step=0, max_depth=8,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [166]:
#11. Metrics 
model.score(x,y)
y_train_predict = model.predict(x)
rmse = (np.sqrt(MSE(y, y_train_predict)))
r2 = r2_score(y, y_train_predict)

print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")



The model performance for training set
--------------------------------------
RMSE is 0.37809406596447737
R2 score is 0.9635095131981214




In [167]:
# model evaluation for testing set

y_test_predict = model.predict(val_X)
# root mean square error of the model
rmse = (np.sqrt(MSE(val_y, y_test_predict)))

# r-squared score of the model
r2 = r2_score(val_y, y_test_predict)

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))

The model performance for testing set
--------------------------------------
RMSE is 1.9647306773710227
R2 score is 0.1522510294199545


In [149]:
#12. Creating Pickle file and generating the output with pickle file
import pickle
filename = 'xgb_model.pkl'
pickle.dump(model, open(filename, 'wb'))


In [179]:
loaded_model = pickle.load(open(filename, 'rb'))
pred_transaction_rev=loaded_model.predict(test_df) # Predicting with Test data
output_df=pd.DataFrame(pred_transaction_rev) # converting predicted values into Dataframe 
output_df.to_csv('predicted_values.csv') # Saving the predicted values into csv file

TypeError: Input data can not be a list.

In [171]:
test_df.drop(['bounces','keyword'],axis=1,inplace=True)

In [170]:
x.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 765707 entries, 0 to 903652
Data columns (total 22 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   channelGrouping  765707 non-null  int64  
 1   visitNumber      765707 non-null  int64  
 2   month            765707 non-null  int64  
 3   year             765707 non-null  int64  
 4   visitHour        765707 non-null  int64  
 5   browser          765707 non-null  int64  
 6   operatingSystem  765707 non-null  int64  
 7   isMobile         765707 non-null  int64  
 8   deviceCategory   765707 non-null  int64  
 9   continent        765707 non-null  int64  
 10  subContinent     765707 non-null  int64  
 11  country          765707 non-null  int64  
 12  region           765707 non-null  int64  
 13  metro            765707 non-null  int64  
 14  city             765707 non-null  int64  
 15  networkDomain    765707 non-null  int64  
 16  hits             765707 non-null  int6

In [175]:
test_df=test_df['newVisits'].astype(int)